# Тема "Эмбеддинг и пословный машинный перевод"

**Задача поиск похожих по эмбеддингам**
Скачиваем датасет 

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

```
# пример работы с ним 
from corus import load_ods_gazeta
path = 'gazeta.csv.gz'
records = load_ods_gazeta(path)
next(records)
```


что надо сделать 

1. на основе word2vec/fasttext реализовать метод поиска ближайших статей
(на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
2. Проверить насколько хорошо работают подходы


ссылка на датасет газеты выдает 404, поэтому я взял датасет ленты

In [30]:
from corus import load_lenta
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [31]:
import pandas as pd

In [32]:
my_list = []
for record in records:
    my_list.append({'url': record.url, 'title': record.title, 'text': record.text, 'topic': record.topic,
                    'tags': record.tags, 'date': record.date})
df = pd.DataFrame(my_list)
df

,url,title,text,topic,tags,date
0,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды,None
1,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир,None
2,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика,None
3,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество,None
4,https://lenta.ru/news/2018/12/15/sobor/,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...,Бывший СССР,Украина,None
...,...,...,...,...,...,...
739345,https://lenta.ru/news/1999/08/31/cholera/,Южно-Сахалинск объявлен очагом холеры,Сегодня областной центр Сахалина и Курил получ...,Россия,Все,None
739346,https://lenta.ru/news/1999/08/31/polit_blok/,Леворадикалы создают предвыборный блок,Бывший шеф Службы безопасности президента Але...,Россия,Все,None
739347,https://lenta.ru/news/1999/08/31/dagestan/,В горах Дагестана идут активные боевые действия,Сегодня утром в районах дагестанских селений Ч...,Россия,Все,None
739348,https://lenta.ru/news/1999/08/31/karachaevo/,Карачаево-Черкесия раскололась по национальном...,Намеченная на сегодняшний день церемония вступ...,Россия,Все,None


In [33]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [34]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

#### Сначала смотрим по заголовкам

In [42]:
target_col = 'title'

In [37]:
assert True

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for index, row in df.iterrows():
    spls = preprocess_txt(row[target_col])
    sentences.append(spls)
    c += 1
    if c > 100000:
        break

In [39]:
sentences = [i for i in sentences if len(i) > 2]

In [40]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

In [55]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for index, row in df.iterrows():
    n_w2v = 0
    n_ft = 0
    spls = row[target_col]
    index_map[index] = spls.replace('\xa0', ' ')
    question = preprocess_txt(spls)

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

In [56]:
def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    print(answers)
    return [index_map[i] for i in answers]

In [57]:
TEXT = "какой город самы красивый"
get_response(TEXT, w2v_index, modelW2V, index_map)

[60584, 60193, 97851, 76280, 96247]


['Москва вошла в число самых дешевых городов Европы для культурного отдыха',
 'Москва заняла одно из последних мест в рейтинге городов с самым доступным жильем',
 'В Москве запустили самый высокий в Европе лифт',
 'Москва вошла в десятку самых привлекательных для ритейлеров городов',
 'Петербург признали городом с самым доступным высшим образованием']

In [59]:
TEXT = "что нового в политике"
get_response(TEXT, w2v_index, modelW2V, index_map)

[9524, 98668, 38888, 25690, 16453]


['У «Натали Турс» возникли новые проблемы',
 'Новым председателем Европарламента избран Антонио Таяни',
 'WADA озаботилось новыми пробирками для допинг-тестов',
 'Болгария поманила Медведева новым газопроводом',
 'В Египте откопали нового Сфинкса']

In [60]:
TEXT = "кто выиграл чемпионат"
get_response(TEXT, w2v_index, modelW2V, index_map)

[40115, 10226, 19183, 82655, 75759]


['СКА досрочно выиграл регулярный чемпионат КХЛ',
 'Определилась страна-хозяйка футбольного чемпионата Европы-2024',
 'Бельгия завоевала бронзу чемпионата мира-2018',
 'СКА выиграл Кубок Гагарина',
 '«Арсенал» выиграл Кубок Англии']

Наблюдение: вроде не так уж плохо

#### Теперь то же самое по тексту статей

In [61]:
target_col = 'text'

In [62]:
assert True

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

for index, row in df.iterrows():
    spls = preprocess_txt(row[target_col])
    sentences.append(spls)
    c += 1
    if c > 100000:
        break
        
sentences = [i for i in sentences if len(i) > 2]

modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=1)
modelFT = FastText(sentences=sentences, vector_size=300, min_count=1, window=5, workers=8)

w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for index, row in df.iterrows():
    n_w2v = 0
    n_ft = 0
    spls = row[target_col]
    index_map[index] = spls.replace('\xa0', ' ')
    question = preprocess_txt(spls)

    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

    if counter > 100000:
        break

w2v_index.build(10)
ft_index.build(10)

True

Наблюдение: на полных текстах модель училась ооочень долго

In [66]:
TEXT = "какой город самый красивый"
get_response(TEXT, w2v_index, modelW2V, index_map)

[88850, 36324, 80755, 47591, 17406]


['Новозеландский город Крайстчерч, находящийся на восточном побережье Южного острова и обновленный после землетрясения в ноябре, признали одной из мировых столиц уличного искусства. Об этом во вторник, 14 марта, сообщает Mashable. Finished wall for #risestreetart @oiyoustreetart #canterburymuseum #christchurch #newzealand #rone #teresaoman Фото опубликовано @r_o_n_e «Современная сцена стрит-арта родилась здесь из трагедии. Граффити стали неотъемлемой частью атмосферы восстановленного города», — цитирует издание путеводитель Lonely Planet, включивший населенный пункт в рейтинг лучших направлений для любителей уличного искусства. Рисунки начали появляться после землетрясения, во время процесса реконструкции и обновления Крайстчерча. Помимо граффити — настенных изображений и надписей — в городе популярны другие виды современного стрит-арта, в том числе уличная скульптура.  Фото опубликовано @spook73 Землетрясение магнитудой 7,8 произошло в Новой Зеландии в полночь 14 ноября 2016 года по м

In [67]:
TEXT = "что нового в политике"
get_response(TEXT, w2v_index, modelW2V, index_map)

[30267, 39012, 49573, 63677, 75394]


['Бывший министр финансов России и глава Центра стратегических разработок (ЦСР) Алексей Кудрин предложил не отвечать на новые санкции со стороны США. Бывший чиновник считает новый российский ответ нецелесообразным, передает ТАСС. «Я считаю, что в отношении частного американского бизнеса санкции нецелесообразны, так как бизнес приносит сюда технологии и инвестиции. Санкции будут ущемлять и нашу экономическую ситуацию, они не будут продуктивны», — сказал Кудрин. Экс-глава Минфина отметил, что вводившие санкции политики — это одно, а бизнес — другое. Глава ЦСР также напомнил слова президента России Владимира Путина на прошлом Петербургском международном экономическом форуме: «Президент звал все иностранные компании. Введение ответных санкций было бы обманом обещаний, которые он дал». Минфин США 6 апреля ввел новые санкции против российских компаний, чиновников, государственных менеджеров и бизнесменов. За первые два торговых дня после введения ограничений российский рынок обвалился, рубль

In [68]:
TEXT = "кто выиграл чемпионат"
get_response(TEXT, w2v_index, modelW2V, index_map)

[86539, 83805, 35531, 20543, 21635]


['Женская сборная России завоевала серебряные медали на чемпионате мира в Пекине. Об этом сообщает корреспондент «Ленты.ру». В финальном поединке, который прошел в воскресенье, россиянки проиграли сборной Канады со счетом 3:8. За российскую команду выступали Анна Сидорова, Маргарита Фомина, Александра Раева и Нкеирука Езех, запасной была Алина Ковалева. Бронзовые медали первенства завоевала сборная Шотландии, в матче за третье место обыгравшая команду Швеции. Чемпионат мира-2017 является частью отбора на Олимпийские игры-2018. Россиянки гарантировали себе участие в Играх в Пхенчхане. Российские керлингистки ранее ни разу не выходили в финал чемпионата мира. В активе команды три бронзовых медали ЧМ (2014, 2015, 2016) и четыре золота чемпионатов Европы (2006, 2012, 2015, 2016).',
 'Сборная России завоевала золотые медали чемпионата Европы для девушек не старше 18 лет. Об этом сообщается в пресс-релизе, поступившем в редакцию «Ленты.ру». В финале соревнований команда Александра Карикова н

Наблюдение: все работает. Тематика подбирается достаточно точно.

Сделаем еще более подробный вывод информации из датафрейма:

In [103]:
def get_response_full(question, index, model, index_map):
    global df
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    for i in answers:
        this_line = df.iloc[[i]]
        print('url: ', this_line['url'].to_list()[0])
        print('title: ', this_line['title'].to_list()[0])
        print('text: ', '\n', this_line['text'].to_list()[0])
        print('topic: ', this_line['topic'].to_list()[0])
        print('tags: ', this_line['tags'].to_list()[0])
        print('date: ', this_line['date'].to_list()[0])
        print('\n')

In [108]:
TEXT = "какой город самый красивый"
print(TEXT, '\n')
get_response_full(TEXT, w2v_index, modelW2V, index_map)

какой город самый красивый 

url:  https://lenta.ru/news/2017/03/14/streetart/
title:  Мировой столицей граффити признали город Крайстчерч в Новой Зеландии
text:  
 Новозеландский город Крайстчерч, находящийся на восточном побережье Южного острова и обновленный после землетрясения в ноябре, признали одной из мировых столиц уличного искусства. Об этом во вторник, 14 марта, сообщает Mashable. Finished wall for #risestreetart @oiyoustreetart #canterburymuseum #christchurch #newzealand #rone #teresaoman Фото опубликовано @r_o_n_e «Современная сцена стрит-арта родилась здесь из трагедии. Граффити стали неотъемлемой частью атмосферы восстановленного города», — цитирует издание путеводитель Lonely Planet, включивший населенный пункт в рейтинг лучших направлений для любителей уличного искусства. Рисунки начали появляться после землетрясения, во время процесса реконструкции и обновления Крайстчерча. Помимо граффити — настенных изображений и надписей — в городе популярны другие виды современного

In [109]:
TEXT = "что нового в политике"
print(TEXT, '\n')
get_response_full(TEXT, w2v_index, modelW2V, index_map)

что нового в политике 

url:  https://lenta.ru/news/2018/04/10/zdravo/
title:  Кудрин призвал оставить санкции США без ответа
text:  
 Бывший министр финансов России и глава Центра стратегических разработок (ЦСР) Алексей Кудрин предложил не отвечать на новые санкции со стороны США. Бывший чиновник считает новый российский ответ нецелесообразным, передает ТАСС. «Я считаю, что в отношении частного американского бизнеса санкции нецелесообразны, так как бизнес приносит сюда технологии и инвестиции. Санкции будут ущемлять и нашу экономическую ситуацию, они не будут продуктивны», — сказал Кудрин. Экс-глава Минфина отметил, что вводившие санкции политики — это одно, а бизнес — другое. Глава ЦСР также напомнил слова президента России Владимира Путина на прошлом Петербургском международном экономическом форуме: «Президент звал все иностранные компании. Введение ответных санкций было бы обманом обещаний, которые он дал». Минфин США 6 апреля ввел новые санкции против российских компаний, чиновник

In [112]:
TEXT = "кто выиграл чемпионат"
print(TEXT, '\n')
get_response_full(TEXT, w2v_index, modelW2V, index_map)

кто выиграл чемпионат 

url:  https://lenta.ru/news/2017/03/26/kerling/
title:  Женская сборная России по керлингу завоевала серебряные медали чемпионата мира
text:  
 Женская сборная России завоевала серебряные медали на чемпионате мира в Пекине. Об этом сообщает корреспондент «Ленты.ру». В финальном поединке, который прошел в воскресенье, россиянки проиграли сборной Канады со счетом 3:8. За российскую команду выступали Анна Сидорова, Маргарита Фомина, Александра Раева и Нкеирука Езех, запасной была Алина Ковалева. Бронзовые медали первенства завоевала сборная Шотландии, в матче за третье место обыгравшая команду Швеции. Чемпионат мира-2017 является частью отбора на Олимпийские игры-2018. Россиянки гарантировали себе участие в Играх в Пхенчхане. Российские керлингистки ранее ни разу не выходили в финал чемпионата мира. В активе команды три бронзовых медали ЧМ (2014, 2015, 2016) и четыре золота чемпионатов Европы (2006, 2012, 2015, 2016).
topic:  Спорт
tags:  Зимние виды
date:  None




In [114]:
TEXT = "новые научные открытыия"
print(TEXT, '\n')
get_response_full(TEXT, w2v_index, modelW2V, index_map)

новые научные открытыия 

url:  https://lenta.ru/news/2017/09/11/biocad_stependii/
title:  Лучшие студенты ИТМО получили стипендии от «Биокада»
text:  
 Магистратура по научной коммуникации «Санкт-Петербургского национального исследовательского университета информационных технологий, механики и оптики» (Университета ИТМО) и компания «Биокад» (BIOCAD) учредили стипендиальную программу для лучших студентов второго года обучения. Об этом говорится в пресс-релизе, поступившем в «Ленту.ру». Первые ежемесячные стипендии в размере 20 тысяч рублей получили магистранты Анастасия Сваровская и Ксения Жирнова. Сваровская в Петербург приехала из Новосибирска, где закончила факультет журналистики Новосибирского государственного университета. Вторая студентка, Жирнова, поступила в магистратуру по научной коммуникации из программы Лазерной техники и биомедицинской техники ИТМО. Родом Ксения из Мурманска. В течение текущего учебного года стипендиатки создадут, а затем и защитят оригинальный коммуникаци